In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Feature Extraction of Training and Testing

In [2]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2'
#####################   Modify above path   #########################

testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

test_image_dir = testpath + "images/"
test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

In [3]:
import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

# Training Feature

In [5]:
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)
name=X.columns

scaler = StandardScaler()
X = scaler.fit_transform(X)



In [6]:
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']
train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)

In [ ]:
X

In [ ]:
y

# Testing Feature

In [ ]:
# dataDir = test_pt_dir
# fiducial_pt_list = []
# filelist = []
# for file in os.listdir(dataDir):
#     filelist.append(file)
# filelist.sort()
# for file in filelist:
#     fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
#     l = []
# for i in range(len(fiducial_pt_list)):
#     if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
#         l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
#     else:
#         l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
# fiducial_pt_list = l


# TestInput = pd.DataFrame(np.zeros((2500, 6006)))
# for i in range(2500):
#     TestInput.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)



In [ ]:
# TestInput

In [ ]:
# TestInput.columns=name
# scaler = StandardScaler()
# TestInput = scaler.fit_transform(TestInput)

In [ ]:
# scaler = StandardScaler()
# TestInput = scaler.fit_transform(TestInput)

# Model Training

## BaselineModel: GBM

In [ ]:
Baseline_model= GradientBoostingClassifier(n_estimators=100  , max_depth= 1,learning_rate=0.1)

In [ ]:
# kfold = KFold(n_splits=5, shuffle=True)
# results = cross_val_score(Baseline_model, X,y, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
Baseline_model.fit(X,y)

In [ ]:
joblib.dump(Baseline_model,'BASELINE_model.m')#please change the path

## Advanced Model: VotingClassifier

In [ ]:
randomforest_model_final=RandomForestClassifier(n_estimators = 100, criterion = 'gini', 
                             random_state = 42, min_samples_leaf=1, max_features='sqrt')

In [ ]:
xgboost_model_final =xgboost.XGBClassifier(max_depth=4,n_estimators=50,learning_rate=0.1,
                       min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005)

In [ ]:
svm_model_final =SVC(C=0.1,decision_function_shape='ovr',degree=2,gamma=0.1,kernel='linear')

In [ ]:
logistic_model_final =LogisticRegression(C=0.01,
                                         dual=False,
                                         fit_intercept=True,
                                         intercept_scaling=1,
                                         max_iter=300,
                                         multi_class='multinomial',
                                         penalty='l2',
                                         solver='newton-cg',
                                         tol=0.0001)

In [ ]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(3000,),
                              learning_rate='adaptive',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [ ]:
Final_model = VotingClassifier(
    estimators=[('rf_clf',randomforest_model_final),
                ('log_clf', logistic_model_final),
                ('svm_clf', svm_model_final),
               ('xgb_clf', xgboost_model_final),
                ('MLP_clf',MLP_model_final)],
    voting='hard',
    weights=[1,1.5,1.25,1,1.25],
    
)

In [ ]:
# kfold = KFold(n_splits=5, shuffle=True)
# results = cross_val_score(Final_model, X,y, cv=kfold)
# print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
Final_model.fit(X,y)

In [ ]:
joblib.dump(Final_model,'FINAL_model.m')#please change the path

# Prediction

In [7]:
# joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/FINAL_model.m')
FINAL_model=joblib.load('FINAL_model.m')

# joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/BASELINE_model.m')
BASELINE_model=joblib.load('BASELINE_model.m')

In [15]:
dataDir = test_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


TestInput = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    TestInput.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)

TestInput.columns=name
scaler = StandardScaler()
TestInput = scaler.fit_transform(TestInput)

In [16]:
predict_Advanced=FINAL_model.predict(TestInput)
predict_Baseline=BASELINE_model.predict(TestInput)

In [21]:
label_prediction = pd.read_csv(filepath+'/data/labels_prediction.csv')
label_prediction=label_prediction.iloc[:,1:4]
label_prediction['Baseline']=predict_Baseline
label_prediction['Advanced']=predict_Advanced
label_prediction.to_csv(filepath+'/data/labels_prediction.csv')

In [19]:
FINAL_model.score(TestInput,y)

0.0416

In [17]:
FINAL_model.score(X,y)

1.0

In [20]:
BASELINE_model.score(TestInput,y)

0.0444

In [18]:
BASELINE_model.score(X,y)

0.74